## Setup

In [1]:
!nvidia-smi

Wed Dec 11 17:21:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:0B:00.0 Off |                    0 |
| N/A   36C    P0              57W / 500W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 63.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.9/686.9 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 75.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 137.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 153.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 178.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 87.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 119.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()
@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic('bash', '', cell.format(**globals()))

In [3]:
%load_ext dotenv
%dotenv

## Configuration

In [4]:
import os

In [ ]:
## CONFIG
NUM_GPUS = 4
HF_TOKEN = os.environ["HF_TOKEN_R"]
IGNORE_PATTERNS = "original/consolidated*"
CONFIG_FILE = "llama_3_1_8b_lora_distributed.yaml"

## MODEL
FT_MODEL_REPO = "multimodalai"
BASE_MODEL_HF_ID = "meta-llama/Llama-3.1-8B"
CLIENT = "resume-critique"
MODEL = "llama3_1_8b"
MODEL_NUMBER = "4"
REV_N = "1"
FT_METHOD = "tt_lora"
MODEL_TYPE = "adapter"

MDATA_ID = f"model_{MODEL_NUMBER}_20k"
REV = f"rev_{REV_N}"
FT_MODEL_NAME = f"{CLIENT}-{MODEL}-{FT_METHOD}-{MDATA_ID}-{MODEL_TYPE}-{REV}"
FT_MODEL_HF_ID = f"multimodalai/{FT_MODEL_NAME}"

## DATASET
TRAINING_DATA = "resume_critique_model_4.jsonl"

## PATH
BASE_MODEL_PATH = "base_model/"
TOKENIZER_PATH = f"{BASE_MODEL_PATH}/original/tokenizer.model"
OUTPUT_MODEL_PATH = f"checkpoint/{FT_MODEL_REPO}/{FT_MODEL_NAME}"
TRAINING_DATA_PATH = f"data/{TRAINING_DATA}"
CONFIG_FILE_PATH = f"config/{CONFIG_FILE}"

## TRACKING
WANDB_GROUP_NAME = CLIENT
RUN_WANDB_NAME = f"run-{FT_MODEL_NAME}"
LOGS_PATH = "logs/"

In [11]:
!mkdir -p {OUTPUT_MODEL_PATH}
!mkdir -p {LOGS_PATH}

## Download Base Model

In [12]:
%%pybash
tune download {BASE_MODEL_HF_ID} --output-dir {BASE_MODEL_PATH} --ignore-patterns {IGNORE_PATTERNS} --hf-token {HF_TOKEN}

Ignoring files matching the following patterns: original/consolidated*


Fetching 16 files: 100%|██████████| 16/16 [01:58<00:00,  7.40s/it]


Successfully downloaded model repo and wrote to the following locations:
/home/ubuntu/Development/interntal-fine-tune-foundry/resume-critique/model_1_rev_3/base_model/special_tokens_map.json
/home/ubuntu/Development/interntal-fine-tune-foundry/resume-critique/model_1_rev_3/base_model/.gitattributes
/home/ubuntu/Development/interntal-fine-tune-foundry/resume-critique/model_1_rev_3/base_model/tokenizer.json
/home/ubuntu/Development/interntal-fine-tune-foundry/resume-critique/model_1_rev_3/base_model/LICENSE
/home/ubuntu/Development/interntal-fine-tune-foundry/resume-critique/model_1_rev_3/base_model/model.safetensors.index.json
/home/ubuntu/Development/interntal-fine-tune-foundry/resume-critique/model_1_rev_3/base_model/tokenizer_config.json
/home/ubuntu/Development/interntal-fine-tune-foundry/resume-critique/model_1_rev_3/base_model/model-00004-of-00004.safetensors
/home/ubuntu/Development/interntal-fine-tune-foundry/resume-critique/model_1_rev_3/base_model/README.md
/home/ubuntu/Develo

## Fine-Tune

In [ ]:
%%pybash
tune run \
    --nproc_per_node {NUM_GPUS} \
    lora_finetune_distributed \
    --config {CONFIG_FILE_PATH} \
    tokenizer.path={TOKENIZER_PATH} \
    checkpointer.checkpoint_dir={BASE_MODEL_PATH} \
    checkpointer.output_dir={OUTPUT_MODEL_PATH} \
    dataset.data_files={TRAINING_DATA_PATH} \
    metric_logger.group={WANDB_GROUP_NAME} \
    metric_logger.name={RUN_WANDB_NAME} \
	output_dir={OUTPUT_MODEL_PATH} \
	metric_logger.log_dir={LOGS_PATH}

Running with torchrun...


W1211 19:00:14.973000 48344 site-packages/torch/distributed/run.py:793] 
W1211 19:00:14.973000 48344 site-packages/torch/distributed/run.py:793] *****************************************
W1211 19:00:14.973000 48344 site-packages/torch/distributed/run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1211 19:00:14.973000 48344 site-packages/torch/distributed/run.py:793] *****************************************
INFO:torchtune.utils._logging:Running LoRAFinetuneRecipeDistributed with resolved config:

batch_size: 8
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: base_model/
  checkpoint_files:
  - model-00001-of-00004.safetensors
  - model-00002-of-00004.safetensors
  - model-00003-of-00004.safetensors
  - model-00004-of-00004.safetensors
  model_type: LLAMA3
  output_dir: checkpo